In [1]:
import urllib2
import time
import pdb
import xlwt
import requests, re
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
%pdb on

Automatic pdb calling has been turned ON


In [15]:
# read url from file (length should be 38594)
with open('url_pool_imac.txt') as f:
    url_pool = f.readlines()
print len(url_pool)
print url_pool[584]


38594
http://www.std.gov.cn/db/search/stdDBDetailed?id=63FADED5A9F450D8E05397BE0A0A2798



In [29]:
output = xlwt.Workbook(encoding = 'utf-8')
table = output.add_sheet('sheet1', cell_overwrite_ok = False)
row_idx = 0

In [30]:
# for each page, find all the urls
# url_pool = ['http://www.std.gov.cn/db/search/stdDBDetailed?id=5DDA8BA3779B18DEE05397BE0A0A95A7']
idx = -1
for url in url_pool[13892:15000]:
    # find all the urls    
    idx += 1
    while True:
        time.sleep(1)
        if idx %100 == 0:
            print idx
        try:
            response = urllib2.urlopen(url)  
        except urllib2.HTTPError, e:
            print str(e.reason)
            continue
        break
    html = response.read()
#     print html
    
#     pdb.set_trace()
    # initialize items to save
    tag_sp, sp = [],[]
    [tc_name, tc_number,tc_pubdate, tc_impdate, stop_date, tc_chistd, tc_interstd, tc_charge, tc_type, draft_unit, drafter, backup_num, backup_file] = [],[],[],[],[],[],[],[],[],[],[],[],[]
  
    # check if the page exists
    if "地方标准" not in html:
        print 'The request page does not exist'
        print url
        html = []
        continue
    
    sp = html.split("\n")
    len_sp = len(sp)
#     print sp
    for i, info in enumerate(sp):
        if i <=200 or i>=500:
            continue
#         print i,info
        # name
        if not tc_name and "container main-body" in info:
#             pdb.set_trace()
            tag_sp = sp[i + 5].split(',sans-serif;">')
            tc_name = tag_sp[-1].strip('</h4>\r')
#             print idx, tc_name
            continue
            
        # standard number
        if not tc_number and "标准号" in info:
#             pdb.set_trace()
            tag_sp = sp[i + 1].split('<dd class="basicInfo-item value">')
            tc_number = tag_sp[-1].strip("</dd>\r")
#             print tc_number
            continue
            
        if not tc_pubdate and "发布日期" in info:
#             pdb.set_trace()
            tag_sp = sp[i + 1].split('<dd class="basicInfo-item value">')
            tc_pubdate = tag_sp[-1].strip("</dd>\r")
#             print tc_pubdate
            continue
        
        if not tc_impdate and "实施日期" in info:
#             tag_sp = sp[i + 1].split('<dd class="basicInfo-item value">')
#             pdb.set_trace()
            tc_impdate = sp[i+2]
#             print tc_impdate
            continue
            
        if not tc_chistd and "中国标准分类号" in info:
#             pdb.set_trace()
            tag_sp = sp[i + 1].split('<dd class="basicInfo-item value">')
            tc_chistd = tag_sp[-1].strip("</dd>\r")
#             print tc_chistd
            continue        
        
        if not tc_interstd and "国际标准分类号" in info:
#             pdb.set_trace()
            tag_sp = sp[i + 2].split('</div> " >')
            tc_interstd = tag_sp[-1].strip('</span>\r')
#             tc_interstd = []
#             for item in tag_sp:
#                 if "</div>" in item:
#                     tc_interstd.append(item.strip("</div>")[0]) 
#             print tc_interstd
            continue

        if not tc_charge and '<dt class="basicInfo-item name">主管部门</dt>' in info:
#             pdb.set_trace()
            tag_sp = sp[i + 1].split('<dd class="basicInfo-item value"><a class="triggers" href="" CD_CODE="1" target="_blank">')
            tc_charge = tag_sp[-1].strip("</a></dd>\r")
#             print tc_charge
            continue      

        if not tc_type and "行业分类" in info:
#             pdb.set_trace()
            tag_sp = sp[i + 1].split('<dd class="basicInfo-item value">')
            tc_type = tag_sp[-1].strip("</dd>\r")
#             print tc_type
            continue    
            
        if not stop_date and "标准状态" in info:
#             pdb.set_trace()
            tag_sp = sp[i + 8].split('">废止')
            stop_date =  tag_sp[-1].strip("</span></a></li>\r")
#             print stop_date
            continue

       
        if not draft_unit and '<h2 class="title-text">起草单位</h2>' in info:
#             pdb.set_trace()
            draft_unit = []
            idx2 = 1
            while '<dd class="basicInfo-item value">' not in sp[i+idx2] and idx2 + i < len_sp:
                idx2 += 1
            while '</div>' not in sp[i+idx2] and idx2 + i < len_sp:
                if '<dd class="basicInfo-item value">' in sp[i+idx2]:
                    tag_sp = sp[i + idx2].split('</span>')
                    draft_unit.append(tag_sp[-1].strip("</a></dd>\r")+'\r')
                idx2 += 1
#             print draft_unit
            continue

        if not drafter and '<h2 class="title-text">起草人</h2>' in info:
#             pdb.set_trace()
            drafter = []
            idx2 = 1
            while '<dd class="basicInfo-item twod"' not in sp[i+idx2] and idx2 + i < len_sp:
                idx2 += 1
            while '</div>' not in sp[i+idx2] and idx2 + i < len_sp:
                if '<dd class="basicInfo-item twod"' in sp[i+idx2]:
                    tag_sp = sp[i + idx2].split('</span>')
                    drafter.append(tag_sp[-1].strip("</dd>\r")+'\r')
                idx2 += 1
#             print drafter
            continue
 
        if not backup_num and "备案号" in info:
#             pdb.set_trace()
            tag_sp = sp[i].split('备案号：')
            backup_num = tag_sp[-1].strip("。</p>\r")
#             print backup_num
            continue
            
        if not backup_file and "备案公告" in info:
#             pdb.set_trace()
            tag_sp = sp[i+3].split("target='_blank'>")
            backup_file = tag_sp[-1].strip("</a>\r")
#             print backup_file
            continue    
            
    
    output_tbl = [tc_name, tc_number,tc_pubdate, tc_impdate, stop_date, tc_chistd, tc_interstd, tc_charge, tc_type, draft_unit, drafter, backup_num, backup_file]
#     pdb.set_trace
#     print output_tbl
    if tc_number:
        # encode unit-8 (chinese)
        [table.write(row_idx, i, output_tbl[i]) for i in range(len(output_tbl))]
        row_idx += 1
        tc_number = []
#         print >> output, "\t".join(output_tbl)
    if idx % 100 == 0:
        output.save("/Volumes/data1/yefei_13892_15000.xls")


0
100
200
300
400
The request page does not exist
http://www.std.gov.cn/db/search/stdDBDetailed?id=5DDA8BA3A03B18DEE05397BE0A0A95A7

500
600
700
800
900
1000
1100


In [43]:
import pandas
df = pandas.read_excel('/Volumes/data1/yefei_localstandard.xls')



In [46]:
output.save("/Volumes/data1/yefei_13892_15000.xls")

In [44]:
# values = df['起草单位'].values
# print values
print df.columns

Index([u'名称', u'标准号', u'发布日期', u'实施日期', u'废止时间', u'中国标准分类号', u'国际标准分类号',
       u'主管部门', u'行业分类', u'起草人', u'备案号', u'备案公告', u'起草单位'],
      dtype='object')


In [45]:
values = df[df.columns[-1]].values
for v in values:
    print v

nan
nan
天津市机动车维修管理处
天津市优耐特汽车电控技术服务有限公司

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
重庆市旅游协会
沙坪坝区旅游局
南川区旅游局
重庆市乡村旅游协会
大渡口区旅游局

重庆市旅游协会
铜梁区旅游局
北碚区旅游局
重庆市乡村旅游协会
九龙坡区旅游局

中国热带农业科学院椰子研究所

海南省旅游发展委员会
海南师范大学

中国热带农业科学院椰子研究所

中国热带农业科学院椰子研究所

中国热带农业科学院椰子研究所
云南省农业科学院热带亚热带经济作物研究所

海南省产品质量监督检验所
定安质量技术监督局

贵研资源(易门)有限公司

云南省德宏热带农业科学研究所

西南林业大学
国家高原湿地研究中心

国家林业局昆明勘察设计院
云南省林业厅野生动植物保护与自然保护区管理处
云南黄连山国家级自然保护区管理局
国家林业局自然保护区及野生动植物西南监测中心
云南林业厅科技教育处

海南省产品质量监督检验所
海南省定安质量技术监督局

云南省公安消防总队
云南天瑞消防检测评价有限公司
云南同晋消防检测评价有限公司
云南君平消防检测评价有限公司
昆明市公安消防支队
云南合信消防检测评价有限公司
云南华铄科技有限公司
云南和亚消防检测评

云南省公安消防总队
云南天瑞消防检测评价有限公司
云南同晋消防检测评价有限公司
云南君平消防检测评价有限公司
昆明市公安消防支队
云南合信消防检测评价有限公司
云南华铄科技有限公司
云南和亚消防检测评

云南省公安消防总队
云南云鹰消防设施维护管理有限公司
昆明都市卫士电子设备有限公司
云南捷星工程有限
昆明华龙智腾科技股份有限公司
昆明福瑞达消防技术咨询服务有限公司
云南合泰消防设施维护管理有限公司

云南省公共保洁与家政行业协会
云南省商务厅

昆明学院旅游学院
云南省餐饮与美食行业协会

阿坝州防雷中心
甘孜州防雷中心
中国民用航空飞行学院绵阳分院
凉山州防雷中心
成都兴业雷安电子有限公司
信息产业电子第十一设计研究院科技工程股份有限公司

四川省早期教育行业协会
四川0～3岁儿童早期发展与教育研究中心
四川省标准化研究院

云南省农